In [ ]:
import pandas as pd
import numpy as np
import spacy
!pip install lexicalrichness
from lexicalrichness import LexicalRichness
!python -m spacy download de_core_news_sm
nlp = spacy.load('de_core_news_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lexicalrichness: filename=lexicalrichness-0.5.1-py3-none-any.whl size=15418 sha256=3021dac6f7a82fdc99976f73d7610f925834ef1e13678a26c3b3068df303511e
  Stored in directory: /root/.cache/pip/wheels/eb/40/d0/053edb84485f223effdbf0f91fc2b6ec6fc6cf2230aadca09a
Successfully built lexicalrichness
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 97.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Countes the number of words per essay

def num_of_words(text):
    count = len(text.split())
    return count

#Lexical Diversity
 # MLTD Score: Measure of Lexical Diversity (McCarthy 2005, McCarthy and Jarvis 2010)
def calculate_lex_richness_MTLD2(text):
    lex = LexicalRichness(text)
    lex_rich_score = lex.mtld()
    return(lex_rich_score)

example1 = "Hilal mag das nicht, deswegen sie rennt."
example2= "Am Montag kündigte Sara Danius, die ständige Vizepräsidentin des Nobel-Preis-Committee für Literatur an der Schwedischen Akademie, während einer öffentlichen Verkündigung in einem Radioprogramm auf Sveriges Radio in Schweden, öffentlich bekannt, dass sie die Bemühungen, Bob Dylan direkt zu erreichen, um den 2016er Nobel-Preis für Literatur zu gewinnen, aufgegeben hatte."


calculate_lex_richness_MTLD2(example1)
num_of_words(example2)



49

In [ ]:

# Sentence Complexity

# Return the number of specified dependency labels found
def sent_complexity_structure(text):
    doc = nlp(text)
    return len([token for token in doc if (token.dep_ == "nk" or token.dep_ == "sb" or token.dep_ == "oc"or token.dep_ == "sbp"
    or token.dep_ == "rc" or token.dep_ == "mo" or token.dep_ == "nmc" or token.dep_ == "ag" or token.dep_ == "cc")])



def calculate_dependency_depth(text):
    doc = nlp(text)
    sentence_depths = {}
    for sent in doc.sents:
        max_depth = 0
        for token in sent:
            depth = 0
            # Traverse ancestors until the root (token.head == token for the root)
            current_token = token
            while current_token != current_token.head:
                depth += 1
                current_token = current_token.head
            if depth > max_depth:
                max_depth = depth
        sentence_depths[sent.text] = max_depth

    return sentence_depths
calculate_dependency_depth(example2)


{'Am Montag kündigte Sara Danius, die ständige Vizepräsidentin des Nobel-Preis-Committee für Literatur an der Schwedischen Akademie, während einer öffentlichen Verkündigung in einem Radioprogramm auf Sveriges Radio in Schweden, öffentlich bekannt, dass sie die Bemühungen, Bob Dylan direkt zu erreichen, um den 2016er Nobel-Preis für Literatur zu gewinnen, aufgegeben hatte.': 8}

In [ ]:
def walk_tree(token, depth):
    max_subtree_depth = depth
    for child in token.children:
        max_subtree_depth = max(max_subtree_depth, walk_tree(child, depth + 1))
    return max_subtree_depth

def calculate_dep_length(text_input):
    # Process the text with spaCy to get a Doc object
    doc = nlp(text_input)
    temp = []
    for sentence in doc.sents:
        # Assuming sentence.root is the root token of the sentence
        temp.append(walk_tree(sentence.root, 0))
    return np.mean(temp)

# Example usage:
my_text = "Am Montag kündigte Sara Danius, die ständige Vizepräsidentin des Nobel-Preis-Committee für Literatur an der Schwedischen Akademie, während einer öffentlichen Verkündigung in einem Radioprogramm auf Sveriges Radio in Schweden, öffentlich bekannt, dass sie die Bemühungen, Bob Dylan direkt zu erreichen, um den 2016er Nobel-Preis für Literatur zu gewinnen, aufgegeben hatte."
mean_dependency_length = calculate_dep_length(my_text)
print(f"Mean dependency length: {mean_dependency_length}")


## DO WE NEED TO NORMALIZE W.R.T NUMBER OF WORDS IN SENTENCE?

Mean dependency length: 8.0


In [ ]:

def calculate_lex_richness_MTLD2(s):
  if len(s) == 0:
    return None
  return LexicalRichness(s).mtld()

def word_count(s):
  if len(s) == 0:
    return None
  return len(s.split())

def tree_depth(token):
    max_subtree_depth = 0
    for child in token.children:
        max_subtree_depth = max(max_subtree_depth, tree_depth(child)+1)
    return max_subtree_depth

def dependency_depth(t):
    doc = nlp(t)
    l = []
    #Usually, only one sent per translation. but llm might output more.
    for s in doc.sents:
        l.append(tree_depth(s.root))
    return np.mean(l)

def sent_complexity_structure(text):
    doc = nlp(text)
    return sum(1 for t in doc if t.dep_ in {"nk", "sb", "oc", "sbp", "rc", "mo", "nmc", "ag", "cc"})

#Coordinating Conjunctions - und, aber, denn, oder, sondern.
def sent_c_conjunctions(text):
   doc = nlp(text)
   return sum(1 for t in doc if t.pos_ in {"CCONJ"})

#subordinating conjunction - dass, weil, denn, ob, als,
def sent_s_conjunctions(text):
   doc = nlp(text)
   return sum(1 for t in doc if t.pos_ in {"SCONJ"})

# contrastive connecting conjunctions - 'aber', 'sondern', 'doch', 'obwohl', 'während', 'jedoch', 'trotzdem'
def sent_contra_conjunctions(text):
   doc = nlp(text)
   contrastive_conjunctions = ['aber', 'sondern', 'doch', 'obwohl', 'während', 'jedoch', 'trotzdem']
   return sum(1 for token in doc if token.text.lower() in contrastive_conjunctions)


def sent_punctuation(text):
   doc = nlp(text)
   return sum(1 for t in doc if t.pos_ in {"PUNCT"})

def sent_commas(text):
   return text.count(',')

def yule_k(s):
  return LexicalRichness(s).yulek


In [ ]:
def all_metrics(path):
  with open(path) as f:
    sentences = f.readlines()
    sentences = [s for s in sentences if s != ""]
    full_text = "\n".join(sentences)
    print ("Yule_k" + str(yule_k(full_text)))
    #print ("Yule_k " + str(np.mean([yule_k(s) for s in sentences])))
    print("lex richness "+  str(calculate_lex_richness_MTLD2(full_text)))
    #print("lex richness "+  str(np.mean([calculate_lex_richness_MTLD2(s) for s in sentences])))
    print("word_count "+  str(word_count(full_text)))
    #print("word_count "+  str(np.mean([word_count(s) for s in sentences])))
    #print("dep depth "+  str(np.mean([dependency_depth(s) for s in sentences])))
    print("C_Conjunctions [de] "+  str(np.sum([sent_c_conjunctions(s) for s in sentences])))
    print("S_Conjunctions [de] "+  str(np.sum([sent_s_conjunctions(s) for s in sentences])))
    print("contra_ Conjunctions [de] "+  str(np.sum([sent_contra_conjunctions(s) for s in sentences])))
    #print("Complexity structure "+  str(np.mean([sent_complexity_structure(s) for s in sentences])))
    print("Punctuation "+  str(np.sum([sent_punctuation(s) for s in sentences])))
    print("Commas "+  str(np.sum([sent_commas(s) for s in sentences])))

In [ ]:
all_metrics("/content/ref.txt")

FileNotFoundError: [Errno 2] No such file or directory: '/content/ref.txt'

In [ ]:
all_metrics("/content/base-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k63.55962628111171
lex richness 159.35939401329767
word_count 21114
C_Conjunctions [de] 866
S_Conjunctions [de] 511
contra_ Conjunctions [de] 158
Punctuation 3291
Commas 1727


In [ ]:
all_metrics("/content/connecting_word-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k61.92563943157043
lex richness 162.67620497820548
word_count 20039
C_Conjunctions [de] 839
S_Conjunctions [de] 394
contra_ Conjunctions [de] 131
Punctuation 2945
Commas 1502


In [ ]:
all_metrics("/content/grammar-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k63.95833481205982
lex richness 70.08622457155087
word_count 21445
C_Conjunctions [de] 836
S_Conjunctions [de] 477
contra_ Conjunctions [de] 139
Punctuation 3218
Commas 1662


In [ ]:
all_metrics("/content/less_common-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k65.57778288915065
lex richness 167.12633231922572
word_count 21439
C_Conjunctions [de] 861
S_Conjunctions [de] 495
contra_ Conjunctions [de] 151
Punctuation 3347
Commas 1724


In [ ]:
all_metrics("/content/literal-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k61.65770253181906
lex richness 167.80610401504867
word_count 21692
C_Conjunctions [de] 834
S_Conjunctions [de] 519
contra_ Conjunctions [de] 151
Punctuation 3478
Commas 1820


In [ ]:
all_metrics("/content/not_natural-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k58.05471203921895
lex richness 188.207516990728
word_count 20919
C_Conjunctions [de] 764
S_Conjunctions [de] 412
contra_ Conjunctions [de] 125
Punctuation 3174
Commas 1618


In [ ]:
all_metrics("/content/robotic-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k59.25720584700894
lex richness 170.90536576867066
word_count 21301
C_Conjunctions [de] 781
S_Conjunctions [de] 458
contra_ Conjunctions [de] 131
Punctuation 3817
Commas 1675


In [ ]:
all_metrics("/content/structure-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k73.87049607211254
lex richness 85.67688660499289
word_count 21815
C_Conjunctions [de] 1070
S_Conjunctions [de] 514
contra_ Conjunctions [de] 158
Punctuation 3330
Commas 1764


In [ ]:
all_metrics("/content/simple-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k63.411125644864924
lex richness 156.32403466223965
word_count 20548
C_Conjunctions [de] 864
S_Conjunctions [de] 510
contra_ Conjunctions [de] 141
Punctuation 3227
Commas 1657


In [ ]:
all_metrics("/content/wmt19-ende-wmtp.ref")

Yule_k49.377706724641314
lex richness 195.953523950697
word_count 45008
C_Conjunctions [de] 1227
S_Conjunctions [de] 956
contra_ Conjunctions [de] 190
Punctuation 6766
Commas 3259


In [ ]:
all_metrics("/content/wmt19.en-de.de")

Yule_k52.03036788378187
lex richness 159.86678321678323
word_count 42087
C_Conjunctions [de] 1296
S_Conjunctions [de] 1163
contra_ Conjunctions [de] 286
Punctuation 6943
Commas 3579


In [ ]:
all_metrics("/content/wmt20-ende-wmtp.ref")

Yule_k51.64907141457407
lex richness 174.50790051189418
word_count 47807
C_Conjunctions [de] 1356
S_Conjunctions [de] 975
contra_ Conjunctions [de] 205
Punctuation 7152
Commas 3477


In [ ]:
all_metrics("/content/wmt20.en-de.de")

Yule_k53.785389515361345
lex richness 161.78237144926302
word_count 45855
C_Conjunctions [de] 1375
S_Conjunctions [de] 1294
contra_ Conjunctions [de] 271
Punctuation 7406
Commas 3865


In [ ]:
all_metrics("/content/ref.txt")

Yule_k60.59015654462974
lex richness 182.3948994223672
word_count 21760
C_Conjunctions [de] 880
S_Conjunctions [de] 473
contra_ Conjunctions [de] 144
Punctuation 3280
Commas 1583


In [ ]:
all_metrics("/content/base-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k63.55962628111171
lex richness 159.35939401329767
word_count 21114
C_Conjunctions [de] 866
S_Conjunctions [de] 511
contra_ Conjunctions [de] 158
Punctuation 3291
Commas 1727


In [ ]:
all_metrics("/content/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k62.453017232633535
lex richness 95.22662397196414
word_count 20979
C_Conjunctions [de] 862
S_Conjunctions [de] 449
contra_ Conjunctions [de] 150
Punctuation 3091
Commas 1532


In [ ]:
all_metrics("/content/direct11-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k66.0661435817144
lex richness 158.61816151628074
word_count 21614
C_Conjunctions [de] 870
S_Conjunctions [de] 514
contra_ Conjunctions [de] 152
Punctuation 3294
Commas 1743


In [ ]:
all_metrics("/content/paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

Yule_k68.87124152653155
lex richness 164.64888582544978
word_count 22682
C_Conjunctions [de] 874
S_Conjunctions [de] 524
contra_ Conjunctions [de] 175
Punctuation 3176
Commas 1706


In [ ]:
all_metrics("/content/persona2-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k71.73626514795963
lex richness 146.39188496242042
word_count 23779
C_Conjunctions [de] 977
S_Conjunctions [de] 551
contra_ Conjunctions [de] 173
Punctuation 3717
Commas 1937


In [ ]:
all_metrics("/content/fewshot1-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k65.05747194826694
lex richness 163.4688535013791
word_count 21367
C_Conjunctions [de] 872
S_Conjunctions [de] 512
contra_ Conjunctions [de] 160
Punctuation 3433
Commas 1795


In [ ]:
all_metrics("/content/misc2-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k68.17829613412123
lex richness 153.598100028903
word_count 21543
C_Conjunctions [de] 877
S_Conjunctions [de] 502
contra_ Conjunctions [de] 158
Punctuation 3328
Commas 1756


In [ ]:
all_metrics("/content/audience1-llama-3.2-3b-instruct-contrastive-prompt.txt")

Yule_k65.98298533742401
lex richness 149.97505536634952
word_count 22910
C_Conjunctions [de] 908
S_Conjunctions [de] 558
contra_ Conjunctions [de] 164
Punctuation 3766
Commas 1900


In [ ]:
import pandas as pd
import numpy as np
import spacy
!pip install lexicalrichness
from lexicalrichness import LexicalRichness


In [ ]:
def lexical(path):
  with open(path) as f:
    sentences = f.readlines()
    sentences = [s for s in sentences if s != ""]
    full_text = "\n".join(sentences)
    #print ("Yule_k " + str(np.mean([yule_k(s) for s in sentences])))
    print("lex richness "+  str(calculate_lex_richness_MTLD2(full_text)))

In [ ]:
#TR
lexical ("/content/ref.txt")

lex richness 811.3970156541226


In [ ]:
lexical ("/content/base-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 16.216035872736633


In [ ]:
lexical ("/content/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 33.02819355769065


In [ ]:
lexical ("/content/direct11-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 15.248509363818602


In [ ]:
lexical ("/content/paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 26.25818573483076


In [ ]:
lexical ("/content/persona2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 14.164321146317818


In [ ]:
lexical ("/content/fewshot1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 14.564915625714441


In [ ]:
lexical ("/content/misc2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 16.213356633966878


In [ ]:
lexical ("/content/audience1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 19.653990770975724


In [ ]:
#CAT

In [ ]:
lexical("/content/ref.txt")

lex richness 89.21086082566376


In [ ]:
lexical("/content/base-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 34.86509518601817


In [ ]:
lexical("/content/direct11-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 46.22096188235034


In [ ]:
lexical("/content/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 96.53053209856012


In [ ]:
lexical("/content/paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 84.8203281240229


In [ ]:
lexical("/content/persona2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 49.28811458136505


In [ ]:
lexical("/content/fewshot1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 39.21264509921417


In [ ]:
lexical("/content/misc2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 89.70255324706989


In [ ]:
lexical("/content/audience1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 47.624933855934245


In [ ]:
#FI

In [ ]:
lexical("/content/ref.txt")

lex richness 1511.8462376697316


In [ ]:
lexical("/content/base-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 19.534486925704822


In [ ]:
lexical("/content/direct11-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 27.916371563281828


In [ ]:
lexical("/content/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 36.98957524380231


In [ ]:
lexical("/content/paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 848.4438625608782


In [ ]:
lexical("/content/persona2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 21.07661687380486


In [ ]:
lexical("/content/fewshot1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 21.909607652554183


In [ ]:
lexical("/content/misc2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 27.26165224973009


In [ ]:
lexical("/content/audience1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 25.734665204888323


In [ ]:
#LIT

In [ ]:
lexical("/content/ref.txt")

lex richness 1008.7397393506828


In [ ]:
lexical("/content/base-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 5.882552668094707


In [ ]:
lexical("/content/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 7.90513747079358


In [ ]:
lexical("/content/direct11-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 6.606014024452648


In [ ]:
lexical("/content/paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 14.306421081266926


In [ ]:
lexical("/content/persona2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 6.977483749703218


In [ ]:
lexical("/content/fewshot1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 6.802117418786284


In [ ]:
lexical("/content/misc2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 6.762054673379652


In [ ]:
lexical("/content/audience1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 6.198894849214325


In [ ]:
#contra - less common

In [ ]:
lexical("/content/less_common_minus02-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 160.35635047520023


In [ ]:
lexical("/content/less_common_minus04-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 159.43029418245555


In [ ]:
lexical("/content/less_common_minus06-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 159.11605973228905


In [ ]:
lexical("/content/less_common_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 163.76833942684627


In [ ]:
#contra - robotic

In [ ]:
lexical("/content/robotic_minus02-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 161.43735423991345


In [ ]:
lexical("/content/robotic_minus04-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 161.97745279099888


In [ ]:
lexical("/content/robotic_minus06-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 169.76698422011333


In [ ]:
lexical("/content/robotic_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 186.31245481444805


In [ ]:
#contra - not natural

In [ ]:
lexical("/not_natural_minus02-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 160.06683284261243


In [ ]:
lexical("/not_natural_minus04-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 163.57485241748225


In [ ]:
lexical("/not_natural_minus06-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 168.60226933713034


In [ ]:
lexical("/not_natural_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 165.17707363803885


In [ ]:
#contra -literal

In [ ]:
lexical("/literal_minus02-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 160.06801836536178


In [ ]:
lexical("/literal_minus04-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 165.90613999292404


In [ ]:
lexical("/literal_minus06-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 167.41284030754338


In [ ]:
lexical("/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 95.22662397196414


In [ ]:
#contra - connect

In [ ]:
lexical("/connecting_minus02-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 158.3180526408761


In [ ]:
lexical("/connecting_minus04-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 160.42574282728265


In [ ]:
lexical("/connecting_minus06-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 119.34967064031252


In [ ]:
lexical("/connecting_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 154.3862270726538


In [ ]:
lexical("/content/paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 164.64888582544978


In [ ]:
lexical("/content/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 95.22662397196414


In [ ]:
# FLORES-DEV EN-DE

In [ ]:
lexical("/content/ref.txt")

lex richness 183.76667948485064


In [ ]:
lexical("/content/base-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 163.04361672793243


In [ ]:
lexical("/content/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 179.78297637772283


In [ ]:
lexical("/content/direct11-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 157.800630054728


In [ ]:
lexical("/content/paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 163.92149877568676


In [ ]:
lexical("/content/persona2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 155.38886129420555


In [ ]:
lexical("/content/fewshot1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 156.91014786601966


In [ ]:
lexical("/content/misc2-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 72.53979996306569


In [ ]:
lexical("/content/audience1-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 149.7228349987921


In [ ]:
lexical("/content/de-direct3-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 166.1801866028377


In [ ]:
lexical("/content/tr-direct3-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 17.09445847924391


In [ ]:
lexical("/content/cat-direct3-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 60.86065918775056


In [ ]:
lexical("/content/fi-direct3-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 19.822671684923577


In [ ]:
lexical("/content/lt-direct3-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 7.335023428959646


In [ ]:
lexical("/content/paraphrase_sys_1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 155.88019146511434


In [ ]:
lexical("/content/paraphrase_sys_2-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 93.24608924765437


In [ ]:
lexical("/content/paraphrase_sys_3-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 153.58030040010172


In [ ]:
lexical("/content/paraphrase_sys_4-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 139.02405125670379


In [ ]:
lexical("/content/lt-paraphrase1-paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 14.306421081266926


In [ ]:
lexical("/content/lt-paraphrase1-src-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 14.306421081266926


In [ ]:
lexical("/content/tr-paraphrase1-paraphrase1-base-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 26.25818573483076


In [ ]:
lexical("/content/tr-paraphrase1-ref-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 26.25818573483076


In [ ]:
lexical("/content/tr-paraphrase1-src-llama-3.2-3b-instruct-paraphrase-prompt.txt")

lex richness 26.25818573483076


In [ ]:
lexical("/content/literal_minus08-llama-3.2-3b-instruct-contrastive-prompt.txt")

lex richness 95.22662397196414
